<a href="https://colab.research.google.com/github/Siddhi2408/Machine_Learning_Journey/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd

df = pd.read_csv("fake_news.csv", encoding="latin1")
df.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0


In [38]:
df.info()
df['labels'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   unit_id          804 non-null    int64 
 1   article_title    804 non-null    object
 2   article_content  804 non-null    object
 3   source           804 non-null    object
 4   date             804 non-null    object
 5   location         804 non-null    object
 6   labels           804 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 44.1+ KB


,count
labels,
1,426
0,378


In [39]:
print(df.columns)



Index(['unit_id', 'article_title', 'article_content', 'source', 'date',
       'location', 'labels'],
      dtype='object')


In [40]:
df.columns = df.columns.str.strip().str.lower()
print(df.columns)




Index(['unit_id', 'article_title', 'article_content', 'source', 'date',
       'location', 'labels'],
      dtype='object')


In [41]:
df = df.drop(columns=['location'])


In [42]:
print(df.columns)


Index(['unit_id', 'article_title', 'article_content', 'source', 'date',
       'labels'],
      dtype='object')


In [43]:
df['text'] = df['article_title'] + " " + df['article_content']


In [44]:
df[['text', 'labels']].head()


,text,labels
0,Syria attack symptoms consistent with nerve ag...,0
1,Homs governor says U.S. attack caused deaths b...,0
2,Death toll from Aleppo bomb attack at least 11...,0
3,Aleppo bomb blast kills six Syrian state TV We...,0
4,29 Syria Rebels Dead in Fighting for Key Alepp...,0


In [45]:
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(r'[^a-zA-Z]', ' ', regex=True)
df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True)


In [46]:
X = df['text']
y = df['labels']


In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words='english',
    max_df=0.8,
    min_df=3,
    ngram_range=(1,2)
)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [49]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000
)

model.fit(X_train_tfidf, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [50]:
y_pred = model.predict(X_test_tfidf)


In [51]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.5590062111801242

Confusion Matrix:
 [[41 31]
 [40 49]]

Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.57      0.54        72
           1       0.61      0.55      0.58        89

    accuracy                           0.56       161
   macro avg       0.56      0.56      0.56       161
weighted avg       0.56      0.56      0.56       161



In [52]:
def predict_news(title, content):
    text = (title + " " + content).lower()
    text = ''.join([c for c in text if c.isalpha() or c == ' '])
    vector = vectorizer.transform([text])
    pred = model.predict(vector)
    return "Fake News 🟥" if pred[0] == 1 else "Real News 🟩"

predict_news(
    "Breaking shocking secret revealed",
    "Anonymous sources claim massive cover up by authorities"
)


'Real News 🟩'